***GENERATED CODE FOR newautoprophet2106 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import pyspark
from pyspark.ml.feature import Binarizer
import json
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationTimeSeriesForecastingMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]['featureList']
        ForecastFE = configObj["FE"]
        featuresSelectedList = [ForecastFE['features']
                                ['timecolumn'], ForecastFE['features']['tocompare']]
        transformedDF = transformationDF.select(
            [c for c in transformationDF.columns if c in featuresSelectedList])
        transformedDF = CleanseData().replaceNullValues(featureData, transformedDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'state', 'transformation_label': 'String Indexer'}], 'feature': 'state', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'AK', 'max': 'WY', 'missing': '0', 'distinct': '26'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'state'}, {'feature_label': 'state', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('state')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'city', 'transformation_label': 'String Indexer'}], 'feature': 'city', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '0.0', 'stddev': 'nan', 'min': '0', 'max': 'YOAKUM', 'missing': '0', 'distinct': '409'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'city'}, {'feature_label': 'city', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('city')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'hospital_subtype', 'transformation_label': 'String Indexer'}], 'feature': 'hospital_subtype', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Childrens Hospitals', 'max': 'Short Term', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'hospital_subtype'}, {'feature_label': 'hospital_subtype', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('hospital_subtype')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'fips_code', 'threshold': 26431.172, 'transformation_label': 'Binarizer'}], 'feature': 'fips_code', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '26431.17', 'stddev': '20285.59', 'min': '0.0', 'max': '56039.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'fips_code'}, {'feature_label': 'fips_code', 'threshold': 26431.172, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('fips_code')

        transformedDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
        df = transformedDF.toPandas()
        return df


***AUTOML FUNCTIONS***

In [ ]:
from prophet import Prophet
import pandas as pd


def driverProphet(df):
    df.columns = ['ds', 'y']
    df['ds'] = df['ds'].astype(str).str[:-6]
    df['ds'] = pd.to_datetime(df['ds'])
    m = Prophet()
    m.fit(df)
    future = m.make_future_dataframe(periods=365)
    forecast = m.predict(future)
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
    m.plot(forecast)
    return m


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run newautoprophet2106Hooks.ipynb
try:
	#sourcePreExecutionHook()

	dateconvertedlplq = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/part-00000-f6dc14d2-9da7-4f94-99a0-c23622219b8b-c000.csv', 'filename': '1718953493dateconverted.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'FilePath': '/Dataprep/testforsize/dateconverted.csv', 'viewFileName': 'dateconverted.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run newautoprophet2106Hooks.ipynb
try:
	#transformationPreExecutionHook()

	newautoprophetfeatureforecast = TransformationTimeSeriesForecastingMain.run(dateconvertedlplq, json.dumps( {"FE": {"functionList": [{"function": "Original"}], "stage_attributes": {}, "featureList": [{"transformationsData": [{}], "feature": "collection_week", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "collection_week"}, {"transformationsData": [{"feature_label": "state", "transformation_label": "String Indexer"}], "feature": "state", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AK", "max": "WY", "missing": "0", "distinct": "26"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "state"}, {"transformationsData": [{"feature_label": "city", "transformation_label": "String Indexer"}], "feature": "city", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "0.0", "stddev": "nan", "min": "0", "max": "YOAKUM", "missing": "0", "distinct": "409"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "city"}, {"transformationsData": [{"feature_label": "hospital_subtype", "transformation_label": "String Indexer"}], "feature": "hospital_subtype", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Childrens Hospitals", "max": "Short Term", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "hospital_subtype"}, {"transformationsData": [{"feature_label": "fips_code", "threshold": 26431.172, "transformation_label": "Binarizer"}], "feature": "fips_code", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "26431.17", "stddev": "20285.59", "min": "0.0", "max": "56039.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "fips_code"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "is_metro_micro", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.82", "stddev": "0.38", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "is_metro_micro"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_beds_7_day_avg", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "196.84", "stddev": "296.38", "min": "0.0", "max": "4004.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_beds_7_day_avg"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "inpatient_beds_used_covid_7_day_avg", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "19.77", "stddev": "40.08", "min": "0.0", "max": "500.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "inpatient_beds_used_covid..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_beds_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1373.16", "stddev": "2075.75", "min": "0.0", "max": "28028.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_beds_7_day_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "all_adult_hospital_beds_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1260.97", "stddev": "1915.5", "min": "0.0", "max": "26999.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "all_adult_hospital_beds_7..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "all_adult_hospital_inpatient_beds_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1006.5", "stddev": "1430.14", "min": "0.0", "max": "12841.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "all_adult_hospital_inpati..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "inpatient_beds_used_covid_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "88.98", "stddev": "141.1", "min": "0.0", "max": "997.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "inpatient_beds_used_covid..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "86.88", "stddev": "137.75", "min": "0.0", "max": "947.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_adult_patients_hosp..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_adult_patients_hospitalized_confirmed_covid_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "78.83", "stddev": "124.48", "min": "0.0", "max": "865.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_adult_patients_hosp..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "inpatient_beds_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "752.97", "stddev": "1200.28", "min": "0.0", "max": "11494.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "inpatient_beds_7_day_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_icu_beds_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "146.99", "stddev": "261.95", "min": "0.0", "max": "1916.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_icu_beds_7_day_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_staffed_adult_icu_beds_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "123.34", "stddev": "198.55", "min": "0.0", "max": "1675.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_staffed_adult_icu_b..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "icu_beds_used_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "147.97", "stddev": "280.75", "min": "0.0", "max": "2837.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "icu_beds_used_7_day_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "8.15", "stddev": "18.64", "min": "0.0", "max": "193.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "staffed_icu_adult_patient..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "staffed_icu_adult_patients_confirmed_covid_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "7.44", "stddev": "16.73", "min": "0.0", "max": "171.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "staffed_icu_adult_patient..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "previous_day_covid_ED_visits_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "76.5", "stddev": "126.43", "min": "0.0", "max": "884.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "previous_day_covid_ED_vis..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "previous_day_admission_adult_covid_suspected_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "7.23", "stddev": "33.59", "min": "0.0", "max": "479.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "previous_day_admission_ad..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "all_pediatric_inpatient_bed_occupied_7_day_avg", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "10.81", "stddev": "38.31", "min": "0.0", "max": "350.7", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "all_pediatric_inpatient_b..."}], "features": {"timecolumn": "collection_week", "tocompare": "total_beds_7_day_sum"}, "dataPercentage": 100, "originalfile": "/FileStore/platform/uploadedSourceFiles/part-00000-f6dc14d2-9da7-4f94-99a0-c23622219b8b-c000.csv", "statFunction": {"function": "Original", "parameter": ""}}}))

	#transformationPostExecutionHook(newautoprophetfeatureforecast)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run newautoprophet2106Hooks.ipynb
try:
	#mlPreExecutionHook()

	model = driverProphet(newautoprophetfeatureforecast)

	#mlPostExecutionHook(model)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
try:
        future = model.make_future_dataframe(periods=30,freq='MS',include_history=False)
        predictedDataFrame = model.predict(future)
        display(model.plot(predictedDataFrame))
        display(model.plot_components(predictedDataFrame))
        display(predictedDataFrame)
except Exception as ex:
    logging.error(ex)

spark.stop()


